In [1]:
import warnings
import polars as pl
import altair as alt

from src.kristi_promin import kristi_promin
from src.me_to_neurazi import me_to_neurazi
from src.alt_friendly import alt_friendly

alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [3]:
df = pl.read_parquet('data/papezstvo.parquet', use_pyarrow=True)

In [5]:
import geopy.distance

def vzdalenost_od_rima(souradnice):
    try:
        return geopy.distance.geodesic((souradnice['latitude'],souradnice['longitude']), {41.893056,12.482778}).kilometers
    except:
        None

df = df.with_columns(pl.col('narozeni_souradnice').map_elements(vzdalenost_od_rima).alias('vzdalenost_od_rima'))

In [7]:
def kategorie_vzdalenosti(km):
    if km <= 50:
        return 1
    elif km <= 500:
        return 3
    else:
        return 4

In [249]:
def procenta_z_frantiska(km):
    return km / 11137.927271 * 30

In [333]:
do_grafu = df.with_columns(
    (pl.col("pontifikat_zacatek") - pl.col("narozeni_datum")).alias("nastupni_vek")
).with_columns(
    pl.col("nastupni_vek").dt.total_days() / 365.25
).with_columns(
    ((pl.col("umrti_datum") - pl.col("pontifikat_zacatek")).dt.total_days() / 365.25).alias("zivot_po_zvoleni")
).with_columns(
    ((pl.col('pontifikat_konec') - pl.col('narozeni_datum')).dt.total_days() / 365.25).alias('konecny_vek')
).with_columns(
    pl.col("vzdalenost_od_rima").map_elements(kategorie_vzdalenosti).alias("vzdalenost_od_rima_kategorie")
).with_columns(
    (pl.col("pontifikat_konec") - pl.col("pontifikat_zacatek")).dt.total_days().alias("delka_pontifikatu")
).with_columns(
    (pl.col("pontifikat_zacatek") + (pl.col("pontifikat_konec") - pl.col("pontifikat_zacatek")) / 2).alias("pontifikat_prostredek")
).with_columns(
    (pl.col("jmeno") + pl.lit(' (#') + pl.col("vatikan_poradi").cast(str) + pl.lit(')')).alias("jmeno_cislo")
).with_columns(
    pl.when(pl.col('jmeno').is_in(svati)).then(pl.lit('svatý')).when(pl.col('jmeno').is_in(blahoslaveni)).then(pl.lit("blahoslavený")).otherwise(None).alias("status")
).with_columns(
    pl.col("vzdalenost_od_rima").map_elements(procenta_z_frantiska).alias("vzdalenost_od_rima_podil")
).with_columns(
    pl.when(
        pl.col("vzdalenost_od_rima") <= 5
    ).then(
        pl.lit("rodák z Říma")
    ).when(
        pl.col("narozeni_zeme").str.contains("Germ")
    ).then(
        pl.lit("Německo")
    ).when(
        pl.col("narozeni_zeme").str.contains("Poland")
    ).then(
        pl.lit("Polsko")
    ).when(
        pl.col("narozeni_zeme").str.contains("Argent")
    ).then(
        pl.lit("Argentina")
    ).alias("odkud")
)

In [283]:
do_grafu.tail(2)

vatikan_poradi,vatikan_jmeno,vatikan_pontifikat_zacatek,vatikan_pontifikat_konec,vatikan_obcanske_jmeno,vatikan_narozeni_misto,vatikan_pontifikat_zacatek_presnost,vatikan_pontifikat_konec_presnost,id,poradi,jmeno,jmeno_en,narozeni_misto,narozeni_datum,narozeni_presnost,krest_datum,umrti_misto,umrti_datum,umrti_presnost,umrti_kategorie,umrti_pricina,pontifikat_zacatek,pontifikat_konec,rodny_jazyk,dalsi_jazyky,povolani,rady,ucastnik,wikihesla,narozeni_misto_kod,narozeni_souradnice,narozeni_zeme,vzdalenost_od_rima,nastupni_vek,zivot_po_zvoleni,konecny_vek,vzdalenost_od_rima_kategorie,delka_pontifikatu,pontifikat_prostredek,jmeno_cislo,status,vzdalenost_od_rima_podil,odkud
i64,str,str,str,str,str,i64,i64,str,u32,str,str,str,date,i64,date,str,date,i64,str,str,date,date,list[str],list[str],list[str],null,list[str],list[str],str,struct[2],str,f64,f64,f64,f64,i64,i64,date,str,str,f64,str
265,"""Benedict XVI""","""2005-03-24""","""2013-01-28""","""Joseph Ratzinger""","""Marktl am Inn (Bavaria)""",2,2,"""Q2494""",264,"""Benedikt XVI.""","""Benedict XVI""","""Birth house of Benedict XVI""",1927-04-16,11,1927-04-16,"""Klášter Panny Marie""",2022-12-31,11,"""přirozená smrt""",null,2005-04-19,2013-02-28,"[""němčina""]","[""němčina"", ""latina"", … ""španělština""]","[""římskokatolický kněz"", ""náboženský spisovatel"", … ""teolog""]",null,"[""Pope's visit to Germany 2011"", ""October 1978 papal conclave"", … ""Konkláve v roce 2005""]","[""acewiki"", ""afwiki"", … ""zhwiki""]","""Q2348968""","{48.254,12.8419}","""Germany""",707.475278,78.009582,17.700205,85.87269,4,2872,2009-03-25,"""Benedikt XVI. (#265)""",null,1.905584,null
266,"""Francis""","""2013-02-19""","""2025-03-21""","""Jorge Mario Bergoglio""","""Buenos Aires (Argentina)""",2,2,"""Q450675""",265,"""František""","""Pope Francis""","""Flores""",1936-12-17,11,1936-12-25,"""Vatikán""",2025-04-21,11,"""přirozená smrt""","""cévní mozková příhoda""",2013-03-13,2025-04-21,"[""španělština""]","[""španělština"", ""italština"", … ""církevní latina""]","[""římskokatolický kněz"", ""chemik"", … ""autor""]",null,"[""Konkláve v roce 2013"", ""Konkláve v roce 2005""]","[""acewiki"", ""afwiki"", … ""zhwiki""]","""Q643907""","{-34.633333,-58.466667}","""Argentina""",11137.927271,76.235455,12.106776,88.342231,4,4422,2019-04-02,"""František (#266)""",null,30.0,null


In [199]:
do_grafu.select(pl.col("vzdalenost_od_rima").max())

vzdalenost_od_rima
f64
11137.927271


In [103]:
do_grafu.tail(40).select(pl.col("narozeni_zeme")).unique()

narozeni_zeme
str
"""Poland"""
"""Argentina"""
"""Kingdom of Italy"""
"""Italy"""
"""Germany"""


In [77]:
dokumenty = pl.concat([pl.read_parquet("data/encykliky.parquet"), pl.read_parquet("data/buly.parquet"), pl.read_parquet("data/konstituce.parquet")])
dokumenty = dokumenty.with_columns(
    pl.when(pl.col("papez").str.contains("Blessed")).then(pl.lit("blahoslavený")).when(pl.col("papez").str.contains("Saint")).then(pl.lit("svatý")).otherwise(None).alias("status")
).with_columns(
    pl.col('papez').str.split('Pope ')
).with_columns(pl.col('papez').list.slice(1).alias('jmeno_dokumenty')).explode('jmeno_dokumenty')

dokumenty = dokumenty.join(do_grafu, left_on='jmeno_dokumenty', right_on='vatikan_jmeno', how='left')
dokumenty = dokumenty.with_columns(
    ((pl.col('datum') - pl.col('narozeni_datum')).alias('stari_pri_vydani')).dt.total_days() / 365.25
)

In [35]:
dokumenty.filter(~pl.col("status").is_null()).select(pl.col(['papez'])).unique()

papez
list[str]
"[""Saint "", ""Paul VI""]"
"[""Blessed "", ""Pius IX""]"
"[""Saint "", ""John XXIII""]"
"[""Saint "", ""John Paul II""]"
"[""Saint "", ""Pius V""]"
"[""Saint "", ""Pius X""]"


In [37]:
svati = ['Pavel VI.','Jan XXIII.','Jan Pavel II.','Pius V.','Pius X.']
blahoslaveni = ['Pius IX.']

In [51]:
for s in svati:
    print(s)
    print(len(df.filter(pl.col("jmeno") == s)))

Pavel VI.
1
Jan XXIII.
1
Jan Pavel II.
1
Pius V.
1
Pius X.
1


In [53]:
for s in blahoslaveni:
    print(s)
    print(len(df.filter(pl.col("jmeno") == s)))

Pius IX.
1


In [253]:
limit = 25

nejstarsi = do_grafu.tail(limit).select(pl.col("pontifikat_zacatek").min())

poslednich_50_zaklad = alt.Chart(
    do_grafu.tail(limit).to_pandas(),
    width=320,
    title=alt.Title(
        f"Život a dílo posledních {limit} papežů"
    )
).mark_bar(

).encode(
    alt.X("pontifikat_zacatek:T", title=None),
    alt.X2("pontifikat_konec:T"),
    alt.Y("nastupni_vek:Q", title="věk", axis=alt.Axis(labelExpr="datum.label + ' let'")),
    alt.Y2("konecny_vek:Q"),
    alt.Color("status:N",
             scale=alt.Scale(range=['#690A05','#B72D0D','#E99A2B']),
             legend=None),
)

poslednich_50_text = alt.Chart(
    do_grafu.tail(limit).filter(pl.col("delka_pontifikatu") >= 2000).to_pandas(), # B16 2872
).mark_text(
    color="#690A05",
    angle=270,
    baseline="middle",
    align="right",
    fontSize=7,
    font="Asap",
    dx=-2,
).encode(
    alt.X("pontifikat_prostredek:T", title=None),
    alt.Y("nastupni_vek:Q", title=None),
    alt.Text("jmeno_cislo:N")
)

poslednich_50 = poslednich_50_zaklad + poslednich_50_text

poslednich_50 = poslednich_50

poslednich_50

alt.LayerChart(...)

In [107]:
from datetime import date

In [157]:
udalosti = pl.DataFrame(
    [
        {'udalost':['Tridentský','koncil'],'zacatek':date(year=1545, month=12, day=13),'konec':date(year=1563,month=12, day=4)},
        {'udalost':['První','vatikánský','koncil'],'zacatek':date(year=1869, month=12, day=8),'konec':date(year=1870,month=9, day=20)},
        {'udalost':['Druhý','vatikánský','koncil'],'zacatek':date(year=1962, month=10, day=1),'konec':date(year=1965,month=12, day=8)}
    ]
).with_columns(pl.lit(20).alias("kam"))

In [153]:
udalosti

udalost,zacatek,konec,kam
str,date,date,i32
"""Tridentský koncil""",1545-12-13,1563-12-04,20
"""První vatikánský koncil""",1869-12-08,1870-09-20,20
"""Druhý vatikánský koncil""",1962-10-01,1965-12-08,20


In [335]:
razeni = ['konstituce','bula','encyklika']

graf_vzdalenost = alt.Chart(do_grafu.filter(pl.col('pontifikat_zacatek') > nejstarsi).to_pandas()).mark_bar(color='#354150',size=3).encode(
    alt.X("pontifikat_prostredek:T", title=None),
    alt.Y("vzdalenost_od_rima_podil:Q", title=None)
)

graf_vzdalenost_text = alt.Chart(do_grafu.filter(pl.col('pontifikat_zacatek') > nejstarsi).filter(~pl.col("odkud").is_null()).to_pandas()).mark_text(
    color='#354150',
    angle=270,
    baseline="middle",
    align="left",
    fontSize=7,
    font="Asap",
    dx=2,
).encode(
    alt.X("pontifikat_prostredek:T", title=None),
    alt.Y("vzdalenost_od_rima_podil:Q", title=None),
    alt.Text("odkud:N")
)


graf_udalosti = alt.Chart(
    udalosti.filter(pl.col("zacatek") > nejstarsi).to_pandas()
).mark_rect(
    color='#879BBB'
).encode(
    alt.X("zacatek:T"),
    alt.X2("konec:T")
)

text_udalosti = alt.Chart(
    udalosti.filter(pl.col("zacatek") > nejstarsi).to_pandas()
).mark_text(
    color='#354150',
    font='Asap',
    size=7.5,
    align='right',
    dx=3
).encode(
    alt.X("zacatek:T"),
    alt.Y("kam:Q"),
    alt.Text("udalost:N")
)

graf_dokumenty = alt.Chart(dokumenty.filter(pl.col('pontifikat_zacatek') > nejstarsi).to_pandas()).mark_point(
    filled=True,
    size=8,
    opacity=1,
    fill='#F1D1BB'
    #stroke='#F1D1BB',
    
).encode(
    alt.X('datum:T'),
    alt.Y('stari_pri_vydani:Q'),
    alt.Shape('dokument:N', legend=None, sort=razeni, 
              scale=alt.Scale(range=['diamond','triangle','square'])),
    alt.Size('dokument:N', scale=alt.Scale(range=[50,2,2]), sort=razeni, legend=None),
#    alt.Opacity('dokument:N')
)

finale = alt.layer(poslednich_50, graf_vzdalenost, graf_vzdalenost_text, graf_dokumenty).configure_view(stroke='transparent').configure_axis(grid=True, gridOpacity=0.6, gridWidth = 0.5, gridColor='#F1D1BB')

finale

alt.LayerChart(...)

In [307]:
df.filter(pl.col('vzdalenost_od_rima') < 20)

vatikan_poradi,vatikan_jmeno,vatikan_pontifikat_zacatek,vatikan_pontifikat_konec,vatikan_obcanske_jmeno,vatikan_narozeni_misto,vatikan_pontifikat_zacatek_presnost,vatikan_pontifikat_konec_presnost,id,poradi,jmeno,jmeno_en,narozeni_misto,narozeni_datum,narozeni_presnost,krest_datum,umrti_misto,umrti_datum,umrti_presnost,umrti_kategorie,umrti_pricina,pontifikat_zacatek,pontifikat_konec,rodny_jazyk,dalsi_jazyky,povolani,rady,ucastnik,wikihesla,narozeni_misto_kod,narozeni_souradnice,narozeni_zeme,vzdalenost_od_rima
i64,str,str,str,str,str,i64,i64,str,u32,str,str,str,date,i64,date,str,date,i64,str,str,date,date,list[str],list[str],list[str],null,list[str],list[str],str,struct[2],str,f64
3,"""Anacletus or Cletus""","""0080-01-01""","""0092-01-01""","""""","""Rome""",1,1,"""Q80450""",3,"""Anaklét""","""Anacletus""","""Řím""",0025-01-01,9,null,"""Řím""",0090-01-01,8,null,null,0079-01-01,0092-01-01,null,"[""novořečtina""]","[""katolický kněz""]",null,null,"[""afwiki"", ""alswiki"", … ""zhwiki""]","""Q220""","{41.893056,12.482778}","""Italy""",0.000053
4,"""Clement""","""0092-01-01""","""0099-01-01""","""""","""Rome""",1,1,"""Q42887""",4,"""Klement I.""","""Clement I""","""Řím""",0100-01-01,7,null,"""Chersonesos""",0099-01-01,9,"""zabití člověka člověkem""","""poprava utopením""",0088-01-01,0099-01-01,null,"[""latina""]","[""teolog"", ""katolický kněz""]",null,null,"[""afwiki"", ""alswiki"", … ""zhwiki""]","""Q220""","{41.893056,12.482778}","""Italy""",0.000053
6,"""Alexander I""","""0109-01-01""","""0119-01-01""","""""","""Rome""",0,0,"""Q80028""",6,"""Alexandr I.""","""Alexander I""","""Řím""",0100-01-01,7,null,"""Řím""",0116-01-01,9,null,"""poprava stětím""",0107-01-01,0110-01-01,null,null,"[""katolický kněz""]",null,null,"[""afwiki"", ""alswiki"", … ""zhwiki""]","""Q220""","{41.893056,12.482778}","""Italy""",0.000053
7,"""Sixtus I""","""0119-01-01""","""0128-01-01""","""""","""Rome""",0,0,"""Q81634""",7,"""Sixtus I.""","""Sixtus I""","""Řím""",0042-01-01,9,null,"""Řím""",0126-01-01,8,null,null,0115-01-01,0120-01-01,null,null,"[""katolický kněz""]",null,null,"[""afwiki"", ""alswiki"", … ""zhwiki""]","""Q220""","{41.893056,12.482778}","""Italy""",0.000053
15,"""Zephyrinus""","""0198-01-01""","""0218-01-01""","""""","""Rome""",1,0,"""Q101306""",15,"""Zefyrinus""","""Zephyrinus""","""Řím""",null,null,null,"""Řím""",0217-12-20,11,null,null,0199-01-01,0217-12-20,"[""latina""]",null,"[""katolický kněz""]",null,null,"[""afwiki"", ""alswiki"", … ""zhwiki""]","""Q220""","{41.893056,12.482778}","""Italy""",0.000053
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
228,"""Urban VII""","""1590-08-15""","""1590-08-27""","""Giambattista Castagna""","""Rome""",2,2,"""Q134337""",227,"""Urban VII.""","""Urban VII""","""Řím""",1521-08-04,11,null,"""Řím""",1590-09-27,11,"""přirozená smrt""","""malárie""",1590-09-15,1590-09-27,null,"[""latina""]","[""katolický kněz"", ""dočasný jáhen"", ""katolický biskup""]",null,"[""papežská konkláve v roce 1585"", ""papežská konkláve v září 1590""]","[""afwiki"", ""alswiki"", … ""zhwiki""]","""Q220""","{41.893056,12.482778}","""Italy""",0.000053
233,"""Paul V""","""1605-04-29""",null,"""Camillo Borghese""","""Rome""",2,2,"""Q132711""",232,"""Pavel V.""","""Paul V""","""Řím""",1552-09-17,11,null,"""Řím""",1621-01-28,11,"""přirozená smrt""","""cévní mozková příhoda""",1605-05-16,1621-01-28,null,"[""italština""]","[""římskokatolický kněz"", ""katolický biskup""]",null,"[""papežská konkláve v březnu 1605"", ""papežská konkláve v květnu 1605""]","[""afwiki"", ""arwiki"", … ""zhwiki""]","""Q220""","{41.893056,12.482778}","""Italy""",0.000053
236,"""Innocent X""","""1644-09-04""",null,"""Giovanni Battista Pamphilj""","""Rome""",2,2,"""Q101266""",235,"""Inocenc X.""","""Innocent X""","""Řím""",1574-05-06,11,null,"""Řím""",1655-01-07,11,null,null,1644-09-15,1655-01-07,null,null,"[""katolický kněz"", ""diplomat""]",null,"[""papežská konkláve v roce 1644""]","[""afwiki"", ""arwiki"", … ""zhwiki""]","""Q220""","{